In [ ]:
using GAMS

In [ ]:
using JuMP

In [ ]:
using DataStructures

In [ ]:
using DataFrames

In [ ]:
using Surrogates

In [ ]:
using Flux

In [ ]:
using Statistics

In [ ]:
using GLPK

# Novel mathematical formulation for the short-term scheduling of batch plants: Example 5

This GAMS Model presents an application of the novel mathematical formulation for the short-term scheduling of batch plants. The proposed formulation by Ierapetritou and Floudas  is used on a literature example by Kondili et al that involves 7 tasks and 6 units (heater, 2 reactors, separator, 2 mixers). The main process involves a series of 
processing tasks i.e. heating, 2 reactions, separation, mixing.

Formulation Source:
Ierapetritou and Floudas (1998) Effective Continuous-Time Formulation for Short-Term Scheduling. 1. Multipurpose Batch Processes. Independent Engineering Chemical Research 37 (4341-4359)

Example Source:
Sundaramoorthy A, Karimi IA. (2005) A simpler better slot-based continuous-time formulation for short-term scheduling in multipurpose batch plants. Chemical Engineering Science 60 (2679–2702)


In [ ]:
H = 8;
P = 100;

# SETS

Define the sets to be utilised in the MILP
Please task note that if a task can be performed in more than one unit then it is considered a different task for each unit it can be performed in.


In [ ]:
I = ["t1","t2","t3","t4","t5","t6","t7","t8","t9","t10","t11"];
J = ["j1","j2","j3","j4","j5","j6"];
N = range(1,P, step = 1);
S = ["s1","s2","s3","s4","s5","s6","s7","s8","s9","s10","s11","s12","s13"];

Jt1 = [J[1]];
Jt2 = [J[1]];
Jt3 = [J[2]];
Jt4 = [J[3]];
Jt5 = [J[2]];
Jt6 = [J[3]];
Jt7 = [J[2]];
Jt8 = [J[3]];
Jt9 = [J[4]];
Jt10 = [J[5]];
Jt11 = [J[6]];

Ij1 = [I[1],I[2]];
Ij2 = [I[3],I[5],I[7]];
Ij3 = [I[4],I[6],I[8]];
Ij4 = [I[9]];
Ij5 = [I[10]];
Ij6 = [I[11]];

Is1 = [I[1]];
Is2 = [I[3],I[4]];
Is3 = [I[1],I[5],I[6]];
Is4 = [I[3],I[4],I[5],I[6],I[9]];
Is5 = [I[5],I[6],I[9]];
Is6 = [I[2],I[9]];
Is7 = [I[9],I[10],I[11]];
Is8 = [I[2]];
Is9 = [I[2],I[7],I[8]];
Is10 = [I[7],I[8],I[10],I[11]];
Is11 = [I[10],I[11]];
Is12 = [I[10],I[11]];
Is13 = [I[7],I[8]];

Ip = I;
Jp = J;
Np = N;


# PARAMETERS
The available information for the process is given below i.e. storage capacity, initial available inventory, batch size and parameters alpha and beta which are the processing time parameters viz. constant and variable terms


In [ ]:
vmin = OrderedDict(
   "t1" => 0,
   "t2" => 0,
   "t3" => 0,
   "t4" => 0,
   "t5" => 0,
   "t6" => 0,
   "t7" => 0,
   "t8" => 0,
   "t9" => 0,
   "t10" => 20,
   "t11" => 20
)


vmax = OrderedDict(
   "t1" => 100,
   "t2" => 100,
   "t3" => 100,
   "t4" => 150,
   "t5" => 100,
   "t6" => 150,
   "t7" => 100,
   "t8" => 150,
   "t9" => 300,
   "t10" => 200,
   "t11" => 200
)


STin = OrderedDict(
     "s1" => 10000,
     "s2" => 10000,
     "s3" => 0,
     "s4" => 0,
     "s5" => 0,
     "s6" => 50,
     "s7" => 50,
     "s8" => 10000,
     "s9" => 0,
     "s10" => 0,
     "s11" => 10000,
     "s12" => 0,
     "s13" => 0
)


STmax = OrderedDict(
     "s1" => 100000,
     "s2" => 100000,
     "s3" => 100,
     "s4" => 100,
     "s5" => 300,
     "s6" => 150,
     "s7" => 150,
     "s8" => 100000,
     "s9" => 150,
     "s10" => 150,
     "s11" => 100000,
     "s12" => 100000,
     "s13" => 100000
)

alpha = OrderedDict(
"t1" => 0.667,
"t2" => 1.333,
"t3" => 1.333,
"t4" => 0.667,
"t5" => 0.667,
"t6" => 2.000,
"t7" => 1.000,
"t8" => 1.333,
"t9" => 1.333,
"t10" => 1.333,
"t11" => 1.333
)

beta = OrderedDict(
"t1" => 0.00667,
"t2" => 0.01333,
"t3" => 0.00889,
"t4" => 0.00667,
"t5" => 0.00445,
"t6" => 0.00667,
"t7" => 0.01000,
"t8" => 0.00667,
"t9" => 0.00667,
"t10" => 0.01330,
"t11" => 0.00889
)

price = OrderedDict(
"s1" => 0,
"s2" => 0,
"s3" => 0,
"s4" => 0,
"s5" => 0,
"s6" => 0,
"s7" => 0,
"s8" => 0,
"s9" => 0,
"s10" => 0,
"s11" => 0,
"s12" => 5,
"s13" => 5
)

demand = OrderedDict(
"s1" => 0,
"s2" => 0,
"s3" => 0,
"s4" => 0,
"s5" => 0,
"s6" => 0,
"s7" => 0,
"s8" => 0,
"s9" => 0,
"s10" => 0,
"s11" => 0,
"s12" => 0,
"s13" => 0
)

#Table p(i,s) 'proportion of state s produced (p(i,s) > 0), consumed (p(i,s) < 0) by task i'
#        s1    s2      s3    s4    s5    s6     s7     s8     s9     s10     s11    s12    s13
#t1      -1            +1                      
#t2                                     -0.25         -0.75   +1
#t3           -1             +1
#t4           -1             +1
#t5                  -0.5   -0.5  +1
#t6                  -0.5   -0.5  +1
#t7                                                           -1     +0.6                  +0.4
#t8                                                           -1     +0.6                  +0.4
#t9                         +0.1   -1   +0.4   +0.5                           
#t10                                           -0.4                  -0.4    -0.2    +1
#t11                                           -0.4                  -0.4    -0.2    +1


rho_table = DataFrame([(I = "t1", s1 = -1, s2 = 0, s3 = +1, s4 = 0, s5 = 0, s6 = 0, s7 = 0, s8 = 0, s9 = 0, s10 = 0, s11 = 0, s12 = 0, s13 = 0 ),
                  (I = "t2", s1 = 0, s2 = 0, s3 = 0, s4 = 0, s5 = 0, s6 = -0.25, s7 = 0, s8 = -0.75, s9 = +1, s10 = 0, s11 = 0, s12 = 0, s13 = 0 ),
                  (I = "t3", s1 = 0, s2 = -1, s3 = 0, s4 = +1, s5 = 0, s6 = 0, s7 = 0, s8 = 0, s9 = 0, s10 = 0, s11 = 0, s12 = 0, s13 = 0 ),
                  (I = "t4", s1 = 0, s2 = -1, s3 = 0, s4 = +1, s5 = 0, s6 = 0, s7 = 0, s8 = 0, s9 = 0, s10 = 0, s11 = 0, s12 = 0, s13 = 0 ),
                  (I = "t5", s1 = 0, s2 = 0, s3 = -0.5, s4 = -0.5, s5 = +1, s6 = 0, s7 = 0, s8 = 0, s9 = 0, s10 = 0, s11 = 0, s12 = 0, s13 = 0 ),
                  (I = "t6", s1 = 0, s2 = 0, s3 = -0.5, s4 = -0.5, s5 = +1, s6 = 0, s7 = 0, s8 = 0, s9 = 0, s10 = 0, s11 = 0, s12 = 0, s13 = 0 ),
                   (I = "t7", s1 = 0, s2 = 0, s3 = 0, s4 = 0, s5 = 0, s6 = 0, s7 = 0, s8 = 0, s9 = -1, s10 = +0.6, s11 = 0, s12 = 0, s13 = +0.4 ),
                   (I = "t8", s1 = 0, s2 = 0, s3 = 0, s4 = 0, s5 = 0, s6 = 0, s7 = 0, s8 = 0, s9 = -1, s10 = +0.6, s11 = 0, s12 = 0, s13 = +0.4 ),
                   (I = "t9", s1 = 0, s2 = 0, s3 = 0, s4 = +0.1, s5 = -1, s6 = +0.4, s7 = +0.5, s8 = 0, s9 = 0, s10 = 0, s11 = 0, s12 = 0, s13 = 0 ),
                   (I = "t10", s1 = 0, s2 = 0, s3 = 0, s4 = 0, s5 = 0, s6 = 0, s7 = -0.4, s8 = 0, s9 = 0, s10 = -0.4, s11 = -0.2, s12 = +1, s13 = 0 ),
                   (I = "t11", s1 = 0, s2 = 0, s3 = 0, s4 = 0, s5 = 0, s6 = 0, s7 = -0.4, s8 = 0, s9 = 0, s10 = -0.4, s11 = -0.2, s12 = +1, s13 = 0 )]);

rho = OrderedDict( (r[:I],states) => r[Symbol(states)] for r in eachrow(rho_table), states in S);

# MODEL DEFINITION

In [ ]:
example5_netprofit = Model(GLPK.Optimizer)

# VARIABLES

In [ ]:
@variables example5_netprofit begin
    w[i in I, n in N], Bin
    y[j in J, n in N], Bin
    bm[i in I, j in J, n in N] >= 0
    d[s in S, n in N] >= 0
    ST[s in S, n in N] >= 0
    Ts[i in I, j in J, n in N] >= 0
    Tf[i in I, j in J, n in N] >= 0
end

# ALLOCATION CONSTRAINTS
The allocation constraints express that at each unit j and at an event point n only one of the tasks that can be performed
in this unit (i.e. i ∈ Ij ) should take place

In [ ]:
@constraints example5_netprofit begin
    allocation1[j in J, n in N; j == J[1]],
      sum(w[i,n] for i in Ij1) == y[j,n]
    allocation2[j in J, n in N; j == J[2]],
      sum(w[i,n] for i in Ij2) == y[j,n]  
    allocation3[j in J, n in N; j == J[3]],
      sum(w[i,n] for i in Ij3) == y[j,n]
    allocation4[j in J, n in N; j == J[4]],
      sum(w[i,n] for i in Ij4) == y[j,n]
    allocation5[j in J, n in N; j == J[3]],
      sum(w[i,n] for i in Ij5) == y[j,n]
    allocation6[j in J, n in N; j == J[4]],
      sum(w[i,n] for i in Ij6) == y[j,n]  

end

# CAPACITY CONSTRAINTS
The capacity constraints specify the minimum or maximum amount of available material for a task to take place in a unit
Since vmin is zero for all of them, the minimum capacity constraint is not defined


In [ ]:
@constraints example5_netprofit begin
    capacity1[i in I, j in Jt1, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity2[i in I, j in Jt2, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity3[i in I, j in Jt3, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity4[i in I, j in Jt4, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity5[i in I, j in Jt5, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity6[i in I, j in Jt6, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity7[i in I, j in Jt7, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity8[i in I, j in Jt8, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]    
    capacity9[i in I, j in Jt9, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity10[i in I, j in Jt10, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity11[i in I, j in Jt11, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]    

end

# STORAGE CONSTRAINTS
This constraint specifies the maximum storage capacity of each material state

In [ ]:
@constraints example5_netprofit begin
    storage[s in S, n in N],
      ST[s,n] <= STmax[s]
end

# MATERIAL BALANCE: INITIAL CONDITIONS
This first set constraints is specifying the initial amount of each material state at event point 1


In [ ]:
@constraints example5_netprofit begin
    matbal1a[s in S, n in N; s == S[1] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is1 for j in Jt1) - d[s,n]
    matbal1b[s in S, n in N; s == S[2] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is2 for j in Jt3) + sum(rho[i,s]*bm[i,j,n] for i in Is2 for j in Jt4) - d[s,n]
    matbal1c[s in S, n in N; s == S[3] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is3 for j in Jt5) + sum(rho[i,s]*bm[i,j,n] for i in Is3 for j in Jt6) - d[s,n]
    matbal1d[s in S, n in N; s == S[4] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is4 for j in Jt5) + sum(rho[i,s]*bm[i,j,n] for i in Is4 for j in Jt6) - d[s,n]
    matbal1e[s in S, n in N; s == S[5] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is5 for j in Jt9) - d[s,n]
    matbal1f[s in S, n in N; s == S[6] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is6 for j in Jt2) - d[s,n]
    matbal1g[s in S, n in N; s == S[7] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is7 for j in Jt10) + sum(rho[i,s]*bm[i,j,n] for i in Is7 for j in Jt11) - d[s,n]
    matbal1h[s in S, n in N; s == S[8] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is8 for j in Jt2) - d[s,n]
    matbal1i[s in S, n in N; s == S[9] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is9 for j in Jt7) + sum(rho[i,s]*bm[i,j,n] for i in Is9 for j in Jt8) - d[s,n]
    matbal1j[s in S, n in N; s == S[10] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is10 for j in Jt10) + sum(rho[i,s]*bm[i,j,n] for i in Is10 for j in Jt11) - d[s,n]
    matbal1k[s in S, n in N; s == S[11] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is11 for j in Jt10) + sum(rho[i,s]*bm[i,j,n] for i in Is11 for j in Jt11) - d[s,n]
    matbal1l[s in S, n in N; s == S[12] && n == N[1]],
      ST[s,n] == STin[s] - d[s,n]
    matbal1m[s in S, n in N; s == S[13] && n == N[1]],
      ST[s,n] == STin[s] - d[s,n]
   
end

# MATERIAL BALANCE: PROCESSED
The second set of material balance performs a mass balnce on each state i.e. the amount of material s consumed at event point n by task i in unit j and the amount of material s produced at event point n-1 by task i in unit j


In [ ]:
@constraints example5_netprofit begin
    matbal2a[s in S, n in 2:length(N); s == S[1]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt1 for i in Is1) - d[s,n]
    matbal2b[s in S, n in 2:length(N); s == S[2]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt3 for i in Is2) + sum(rho[i,s]*bm[i,j,n] for j in Jt4 for i in Is2) - d[s,n]
    matbal2c[s in S, n in 2:length(N); s == S[3]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt5 for i in Is3) + sum(rho[i,s]*bm[i,j,n] for j in Jt6 for i in Is3) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt1 for i in Is3) - d[s,n]
    matbal2d[s in S, n in 2:length(N); s == S[4]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt5 for i in Is4) + sum(rho[i,s]*bm[i,j,n] for j in Jt6 for i in Is4) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt3 for i in Is4) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt4 for i in Is4) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt9 for i in Is4)- d[s,n]
    matbal2e[s in S, n in 2:length(N); s == S[5]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt9 for i in Is5) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt5 for i in Is5) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt6 for i in Is5) - d[s,n]
    matbal2f[s in S, n in 2:length(N); s == S[6]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt2 for i in Is6) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt9 for i in Is6) - d[s,n]
    matbal2g[s in S, n in 2:length(N); s == S[7]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt10 for i in Is7) + sum(rho[i,s]*bm[i,j,n] for j in Jt11 for i in Is7) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt9 for i in Is7) - d[s,n]
    matbal2h[s in S, n in 2:length(N); s == S[8]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt2 for i in Is8) - d[s,n]
    matbal2i[s in S, n in 2:length(N); s == S[9]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt7 for i in Is9) + sum(rho[i,s]*bm[i,j,n] for j in Jt8 for i in Is9)+ sum(rho[i,s]*bm[i,j,n-1] for j in Jt2 for i in Is9) - d[s,n]
    matbal2j[s in S, n in 2:length(N); s == S[10]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt10 for i in Is10) + sum(rho[i,s]*bm[i,j,n] for j in Jt11 for i in Is10) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt7 for i in Is10) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt8 for i in Is10) - d[s,n]
    matbal2k[s in S, n in 2:length(N); s == S[11]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt10 for i in Is11) + sum(rho[i,s]*bm[i,j,n] for j in Jt11 for i in Is11) - d[s,n]
    matbal2l[s in S, n in 2:length(N); s == S[12]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt10 for i in Is12) + sum(rho[i,s]*bm[i,j,n] for j in Jt11 for i in Is12) - d[s,n]
    matbal2m[s in S, n in 2:length(N); s == S[13]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt7 for i in Is13) + sum(rho[i,s]*bm[i,j,n] for j in Jt8 for i in Is13) - d[s,n]
   
end

# DURATION CONSTRAINTS
These constraints don't only specify how long task i will take in unit j but also specifies the dependence of the duration
on the amount of material to be processed by task i in unit j.

In [ ]:
@constraints example5_netprofit begin
    duration1[i in I, j in Jt1, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration2[i in I, j in Jt2, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration3[i in I, j in Jt3, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration4[i in I, j in Jt4, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration5[i in I, j in Jt5, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration6[i in I, j in Jt6, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration7[i in I, j in Jt7, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration8[i in I, j in Jt8, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration9[i in I, j in Jt9, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration10[i in I, j in Jt10, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration11[i in I, j in Jt11, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]

end

# SEQUENCE CONSTRAINTS
# i. Same Task in Same Unit
The first set of sequence specify that the start of task i at event point n+1 should start after the end of event point n
for the same task performed in unit j.

In [ ]:
@constraints example5_netprofit begin
    sequence1a[i in I, j in Jt1, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] -  H*(2 - w[i,n]- y[j,n])
    sequence1b[i in I, j in Jt2, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1c[i in I, j in Jt3, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1d[i in I, j in Jt4, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1e[i in I, j in Jt5, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] -  H*(2 - w[i,n]- y[j,n])
    sequence1f[i in I, j in Jt6, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1g[i in I, j in Jt7, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1h[i in I, j in Jt8, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1i[i in I, j in Jt9, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1j[i in I, j in Jt10, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1k[i in I, j in Jt11, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])

    sequence2a[i in I, j in Jt1, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2b[i in I, j in Jt2, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2c[i in I, j in Jt3, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2d[i in I, j in Jt4, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2e[i in I, j in Jt5, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2f[i in I, j in Jt6, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2g[i in I, j in Jt7, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2h[i in I, j in Jt8, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2i[i in I, j in Jt9, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2j[i in I, j in Jt10, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2k[i in I, j in Jt11, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]

    sequence3a[i in I, j in Jt1, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3b[i in I, j in Jt2, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3c[i in I, j in Jt3, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3d[i in I, j in Jt4, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3e[i in I, j in Jt5, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3f[i in I, j in Jt6, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3g[i in I, j in Jt7, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3h[i in I, j in Jt8, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3i[i in I, j in Jt9, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3j[i in I, j in Jt10, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3k[i in I, j in Jt11, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]

end

# ii. Different Task in Same Unit
The following constraints establishes the relationship between the starting time of task i at point n+1 and the end time of task i′ (ip) at event point n when different tasks take place in the same unit.

In [ ]:
@constraints example5_netprofit begin
    sequence4a[i in Ij1, ip in Ij1, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
    sequence4b[i in Ij2, ip in Ij2, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
    sequence4c[i in Ij3, ip in Ij3, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
    sequence4d[i in Ij4, ip in Ij4, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
    sequence4e[i in Ij5, ip in Ij5, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
    sequence4f[i in Ij6, ip in Ij6, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])

end

# iii. Different Task in Different Unit
When different tasks i and i′ are performed in different units j and j′ but take place one after the other according to the production recipe. These constraints specify the order in which then tasks in each unit should be performed i.e. heating then reaction 1 then reaction 2 (reaction 2 can't be completed until both reaction 1 and heating are completed) then  separation, heating 2 then reaction 3 then mixing.


In [ ]:
@constraints example5_netprofit begin
    sequence5a[i in I, ip in I, j in Jt3, jp in J, n in 1:length(N)-1; i == I[3] && ip == I[1] && jp == J[1]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5b[i in I, ip in I, j in Jt4, jp in J, n in 1:length(N)-1; i == I[4] && ip == I[1] && jp == J[1]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5c[i in I, ip in I, j in Jt5, jp in J, n in 1:length(N)-1; i == I[5] && ip == I[3] && jp == J[2]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5d[i in I, ip in I, j in Jt6, jp in J, n in 1:length(N)-1; i == I[6] && ip == I[3] && jp == J[2]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5e[i in I, ip in I, j in Jt5, jp in J, n in 1:length(N)-1; i == I[5] && ip == I[4] && jp == J[3]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5f[i in I, ip in I, j in Jt6, jp in J, n in 1:length(N)-1; i == I[6] && ip == I[4] && jp == J[3]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5g[i in I, ip in I, j in Jt9, jp in J, n in 1:length(N)-1; i == I[9] && ip == I[5] && jp == J[2]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5h[i in I, ip in I, j in Jt9, jp in J, n in 1:length(N)-1; i == I[9] && ip == I[6] && jp == J[3]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5i[i in I, ip in I, j in Jt10, jp in J, n in 1:length(N)-1; i == I[10] && ip == I[9] && jp == J[4]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5j[i in I, ip in I, j in Jt11, jp in J, n in 1:length(N)-1; i == I[11] && ip == I[9] && jp == J[4]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5k[i in I, ip in I, j in Jt7, jp in J, n in 1:length(N)-1; i == I[7] && ip == I[2] && jp == J[1]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5l[i in I, ip in I, j in Jt8, jp in J, n in 1:length(N)-1; i == I[8] && ip == I[2] && jp == J[1]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5m[i in I, ip in I, j in Jt10, jp in J, n in 1:length(N)-1; i == I[10] && ip == I[7] && jp == J[2]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5n[i in I, ip in I, j in Jt11, jp in J, n in 1:length(N)-1; i == I[11] && ip == I[7] && jp == J[2]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5o[i in I, ip in I, j in Jt10, jp in J, n in 1:length(N)-1; i == I[10] && ip == I[8] && jp == J[3]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5p[i in I, ip in I, j in Jt11, jp in J, n in 1:length(N)-1; i == I[10] && ip == I[8] && jp == J[3]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5q[i in I, ip in I, j in Jt7, jp in J, n in 1:length(N)-1; i == I[7] && ip == I[10] && jp == J[5]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5r[i in I, ip in I, j in Jt8, jp in J, n in 1:length(N)-1; i == I[8] && ip == I[10] && jp == J[5]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5s[i in I, ip in I, j in Jt7, jp in J, n in 1:length(N)-1; i == I[7] && ip == I[11] && jp == J[6]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5t[i in I, ip in I, j in Jt8, jp in J, n in 1:length(N)-1; i == I[8] && ip == I[11] && jp == J[6]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])

end

# iv. Completion of previous tasks
A task i' (ip) performed in unit j cannot start until task i in unit j is completed


In [ ]:
@constraints example5_netprofit begin
   sequence6a[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence6b[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence6c[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence6d[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)
   sequence6e[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij5)
   sequence6f[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij6)

   sequence7a[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence7b[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence7c[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence7d[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)
   sequence7e[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij5)
   sequence7f[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij6)

   sequence8a[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence8b[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence8c[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence8d[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)
   sequence8e[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij5)
   sequence8f[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij6)

   sequence9a[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence9b[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence9c[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence9d[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)
   sequence9e[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij5)
   sequence9f[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij6)

   sequence10a[i in I, j in Jt5, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence10b[i in I, j in Jt5, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence10c[i in I, j in Jt5, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence10d[i in I, j in Jt5, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)
   sequence10e[i in I, j in Jt5, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij5)
   sequence10f[i in I, j in Jt5, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij6)

   sequence11a[i in I, j in Jt6, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence11b[i in I, j in Jt6, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence11c[i in I, j in Jt6, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence11d[i in I, j in Jt6, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)
   sequence11e[i in I, j in Jt6, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij5)
   sequence11f[i in I, j in Jt6, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij6)

   sequence12a[i in I, j in Jt7, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence12b[i in I, j in Jt7, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence12c[i in I, j in Jt7, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence12d[i in I, j in Jt7, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)
   sequence12e[i in I, j in Jt7, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij5)
   sequence12f[i in I, j in Jt7, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij6)

   sequence13a[i in I, j in Jt8, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence13b[i in I, j in Jt8, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence13c[i in I, j in Jt8, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence13d[i in I, j in Jt8, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)
   sequence13e[i in I, j in Jt8, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence13f[i in I, j in Jt8, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)

   sequence14a[i in I, j in Jt9, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence14b[i in I, j in Jt9, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence14c[i in I, j in Jt9, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence14d[i in I, j in Jt9, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)
   sequence14e[i in I, j in Jt9, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij5)
   sequence14f[i in I, j in Jt9, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij6)

   sequence15a[i in I, j in Jt10, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence15b[i in I, j in Jt10, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence15c[i in I, j in Jt10, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence15d[i in I, j in Jt10, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)
   sequence15e[i in I, j in Jt10, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij5)
   sequence15f[i in I, j in Jt10, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij6)

   sequence16a[i in I, j in Jt11, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence16b[i in I, j in Jt11, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence16c[i in I, j in Jt11, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence16d[i in I, j in Jt11, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)
   sequence16e[i in I, j in Jt11, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij5)
   sequence16f[i in I, j in Jt11, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij6)

end

# TIME HORIZON CONSTRAINTS
Specify that all tasks should task place within the time horizon


In [ ]:
@constraints example5_netprofit begin
    timehorizon1[i in I, j in J, n in N],
      Ts[i,j,n] <= H
    timehorizon2[i in I, j in J, n in N],
      Tf[i,j,n] <= H
    
end

# OBJECTIVE FUNCTION

## Profit maximization

In [ ]:
@objective example5_netprofit Max begin
    sum(price[s]*d[s,n] for n in N for s in S if (s in [S[12], S[13]]))
end

In [ ]:
optimize!(example5_netprofit);

# Optimal value and Schedule

In [ ]:
println("Optimal Solution: ", JuMP.objective_value(example5_netprofit))
println("Optimal Schedule: ")
println(JuMP.value.(w))
println(JuMP.value.(bm))
println(JuMP.value.(ts))
println(JuMP.value.(tf))